# Working with the Bag-of-Words representation

The [bow module](api.rst#tmtoolkit-bow) in tmtoolkit contains several functions for working with Bag-of-Words (BoW) representations of documents. It's divided into two sub-modules: [bow.bow_stats](api.rst#module-tmtoolkit.bow.bow_stats) and [bow.dtm](api.rst#module-tmtoolkit.bow.dtm). The former implements several statistics and transformations for BoW representations, the latter contains functions to create and convert sparse or dense document-term matrices (DTMs).

Most of the functions in both sub-modules accept and/or return DTMs. The [previous chapter](preprocessing.iypnb) contained a section about what *sparse* DTMs are and [how they can be generated with tmtoolkit](preprocessing.iypnb#Generating-a-sparse-document-term-matrix-(DTM)).

## An example document-term matrix

Before we start with the [bow.dtm](api.rst#module-tmtoolkit.bow.dtm) module, we will generate a sparse DTM from a small example corpus.

In [1]:
import random
random.seed(20191113)   # to make the sampling reproducible

from tmtoolkit.corpus import Corpus

corpus = Corpus.from_builtin_corpus('english-NewsArticles').sample(5)

Let's have a look at a sample document:

In [2]:
print(corpus['NewsArticles-2058'][:227])

Merkel: 'Only if Europe is doing well, will Germany be doing well'

Ahead of meeting her fellow European leaders at a summit in Brussels, German Chancellor Angela Merkel has reiterated her government's call for unity in the EU.


We employ a preprocessing pipeline that removes a lot of information from our original data in order to obtain a very condensed DTM.

In [3]:
from tmtoolkit.preprocess import TMPreproc

preproc = TMPreproc(corpus)
preproc.pos_tag() \
    .lemmatize() \
    .filter_for_pos('N') \
    .tokens_to_lowercase() \
    .remove_special_chars_in_tokens() \
    .clean_tokens(remove_shorter_than=2) \
    .remove_common_tokens(5, absolute=True) # remove tokens that occur in all documents
preproc.tokens_datatable

,doc,position,token,meta_pos
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,NewsArticles-119,0,nhs,NNP
1,NewsArticles-119,1,nhs,NNP
2,NewsArticles-119,2,pledge,NN
3,NewsArticles-119,3,prime,NNP
4,NewsArticles-119,4,minister,NNP
5,NewsArticles-119,5,david,NNP
6,NewsArticles-119,6,cameron,NNP
7,NewsArticles-119,7,theresa,NNP
8,NewsArticles-119,8,may,NNP


In [4]:
preproc.n_docs, len(preproc.vocabulary)

(5, 530)

We fetch the document labels and vocabulary and convert them to NumPy arrays, because such arrays allow advanced indexing methods such as boolean indexing.

In [5]:
doc_labels = np.array(preproc.doc_labels)
doc_labels

array(['NewsArticles-119', 'NewsArticles-1206', 'NewsArticles-2058',
       'NewsArticles-3016', 'NewsArticles-3665'], dtype='<U17')

In [6]:
vocab = np.array(preproc.vocabulary)
vocab[:10]  # only showing the first 10 tokens here

array(['abuse', 'access', 'accession', 'accusation', 'act', 'addition',
       'address', 'addressing', 'administration', 'affiliation'],
      dtype='<U20')

Finally, we fetch the sparse DTM:

In [7]:
dtm = preproc.dtm
dtm

<5x530 sparse matrix of type '<class 'numpy.int32'>'
	with 596 stored elements in Compressed Sparse Row format>

We now have a sparse DTM `dtm`, a list of document labels `doc_labels` that represent the rows of the DTM and a list of vocabulary tokens `vocab` that represent the columns of the DTM. We will use this data for the remainder of the chapter.

## The `bow.dtm` module

This module is quite small. There are two functions to convert a DTM to a [datatable](https://github.com/h2oai/datatable/) or [DataFrame](https://pandas.pydata.org/): [dtm_to_datatable()](api.rst#tmtoolkit.bow.dtm.dtm_to_datatable) and [dtm_to_dataframe()](api.rst#tmtoolkit.bow.dtm.dtm_to_dataframe). Note that the generated datatable or DataFrame is *dense*, i.e. it uses up (much) more memory than the input DTM.

Let's generate a datatable via [dtm_to_datatable()](api.rst#tmtoolkit.bow.dtm.dtm_to_datatable) from our DTM, the document labels and the vocabulary:

In [8]:
from tmtoolkit.bow.dtm import dtm_to_datatable

dtm_to_datatable(dtm, doc_labels, vocab)

,_doc,abuse,access,accession,accusation,act,addition,address,addressing,administration,…,year,yes,york,yucel,yucellast
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,NewsArticles-119,0,1,0,0,0,0,0,0,0,…,0,0,0,0,0
1,NewsArticles-1206,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0
2,NewsArticles-2058,0,0,1,1,0,0,0,1,0,…,2,1,0,1,1
3,NewsArticles-3016,1,0,0,0,0,0,0,0,0,…,0,0,1,0,0
4,NewsArticles-3665,0,1,0,0,1,1,1,0,1,…,1,0,0,0,0


We can see that a row `_doc` with the document labels was created and that the vocabulary tokens become the column names. [dtm_to_dataframe()](api.rst#tmtoolkit.bow.dtm.dtm_to_dataframe) works the same way.

You can combine tmtoolkit with [Gensim](https://radimrehurek.com/gensim/). The `bow.dtm` module provides several functions to convert data between both packages:

- [dtm_and_vocab_to_gensim_corpus_and_dict()](api.rst#tmtoolkit.bow.dtm.dtm_and_vocab_to_gensim_corpus_and_dict): converts a (sparse) DTM and a vocabulary list to a *Gensim Corpus* and *Gensim Dictionary*
- [dtm_to_gensim_corpus()](api.rst#tmtoolkit.bow.dtm.dtm_to_gensim_corpus): convert a (sparse) DTM only to a *Gensim Corpus*
- [gensim_corpus_to_dtm()](api.rst#tmtoolkit.bow.dtm.gensim_corpus_to_dtm): converts a *Gensim Corpus* object to a sparse DTM in COO format

## The `bow.bow_stats` module

This module provides several statistics and transformations for sparse or dense DTMs.

### Document lengths, document and term frequencies, token co-occurrences

Let's start with the [doc_lengths()](api.rst#tmtoolkit.bow.bow_stats.doc_lengths) function, which simply gives the number of tokens per document (i.e. the row-wise sum of the DTM):

In [9]:
from tmtoolkit.bow.bow_stats import doc_lengths

doc_lengths(dtm)

array([ 36,  37, 338, 164, 349])

The returned array is aligned to the document labels `doc_labels` so we can see that the last document, "NewsArticles-3665", is the one with the most tokens. Or to do it computationally:

In [10]:
doc_labels[doc_lengths(dtm).argmax()]

'NewsArticles-3665'

While [doc_lengths()](api.rst#tmtoolkit.bow.bow_stats.doc_lengths) gives the row-wise sum across the DTM, [term_frequencies()](api.rst#tmtoolkit.bow.bow_stats.term_frequencies) gives the column-wise sum. This means it returns an array of the length of the vocabulary's size where each entry in that array reflects the number of occurrences of the respective vocabulary token (aka term).

Let's calculate that measure, get its maximum and the vocabulary token(s) for that maximum value:

In [11]:
from tmtoolkit.bow.bow_stats import term_frequencies

term_freq = term_frequencies(dtm)
(term_freq.max(), vocab[term_freq == term_freq.max()])

(24, array(['medium'], dtype='<U20'))

It's also possible to calculate the proportional frequency, i.e. normalize the counts by the overall number of tokens via `proportions=True`:

In [12]:
term_prop = term_frequencies(dtm, proportions=True)
vocab[term_prop >= 0.01]

array(['candidate', 'eu', 'macron', 'medium', 'merkel', 'refugee'],
      dtype='<U20')

The function [doc_frequencies()](api.rst#tmtoolkit.bow.bow_stats.doc_frequencies) returns how often each token in the vocabulary occurs at least *n* times per document. You can control *n* per parameter `min_val` which is set to `1` by default. The returned array is aligned with the vocabulary. Here, we calculate the document frequency with the default value `min_val=1`, extract the maximum document frequency and see which of tokens in the `vocab` array reach the maximum document frequency:

In [13]:
from tmtoolkit.bow.bow_stats import doc_frequencies

df = doc_frequencies(dtm)
max_df = df.max()
max_df, vocab[df == max_df]

(4, array(['minister'], dtype='<U20'))

It turns out that the maximum document frequency is 4 and only the token "minister" reaches that document frequency. This means only "minister" is mentioned across 4 documents at least once (because `min_val` is `1`). Remember that during preprocessing, we removed all tokens that occur across *all* five documents, hence there can't be a vocabulary token with a document frequency of 5.

Let's see which vocabulary tokens occur within a single document at least 10 times:

In [14]:
df = doc_frequencies(dtm, min_val=10)
vocab[df > 0]

array(['candidate', 'eu', 'macron', 'medium', 'merkel', 'refugee'],
      dtype='<U20')

We can also calculate the *co-document frequency* or *token co-occurrence* matrix via [codoc_frequencies()](api.rst#tmtoolkit.bow.bow_stats.codoc_frequencies). This measures how often each pair of vocabulary tokens occurs at least *n* times together in the same document. Again, you can control *n* per parameter `min_val` which is set to `1` by default. The result is a sparse matrix of shape *vocabulary size* by *vocabulary size*. The columns and rows give the pairs of tokens from the vocabulary.

Let's generate a co-document frequency matrix and convert it to a dense representation, because our further operations don't support sparse matrices.

A co-document frequency matrix is symmetric along the diagonal, because co-occurrence between a pair `(token1, token2)` is always the same as between `(token2, token1)`. We want to filter out the duplicate pairs and for that use [np.triu()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.triu.html) to take only the upper triangle of the matrix, i.e. set all values in the lower triangle including the matrix diagonal to zero (`k=1` does this):

In [15]:
from tmtoolkit.bow.bow_stats import codoc_frequencies

codoc_mat = codoc_frequencies(dtm).todense()
codoc_upper = np.triu(codoc_mat, k=1)
codoc_upper

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Now we create a list that contains the pairs of tokens that occur together in at least two documents (`codoc_upper > 1`) together with their co-document frequency:

In [16]:
interesting_pairs = [(vocab[t1], vocab[t2], codoc_upper[t1, t2])
                     for t1, t2 in zip(*np.where(codoc_upper > 1))]
interesting_pairs[:10]  # showing only the first ten pairs

[('access', 'channel', 2),
 ('access', 'day', 2),
 ('access', 'minister', 2),
 ('access', 'news', 2),
 ('anyone', 'attack', 2),
 ('anyone', 'brexit', 2),
 ('anyone', 'comment', 2),
 ('anyone', 'consensus', 2),
 ('anyone', 'country', 2),
 ('anyone', 'deal', 2)]

### Generate sorted lists and datatables according to term frequency

When working with DTMs, it's often helpful to rank terms per document according to their frequency. This is what [sorted_terms()](api.rst#tmtoolkit.bow.bow_stats.sorted_terms) does for you. It further allows to specify the sorting order (the default is descending order via `ascending=False`) and several limits:

- `lo_thresh` for the minimum term frequency
- `hi_thresh` for the maximum term frequency
- `top_n` for the maximum number of terms per document

Let's display the top three tokens per document by frequency:

In [17]:
from tmtoolkit.bow.bow_stats import sorted_terms

sorted_terms(dtm, vocab, top_n=3)

[[('derbyshire', 2), ('bbc', 2), ('victoria', 2)],
 [('car', 3), ('collision', 3), ('road', 3)],
 [('merkel', 14), ('eu', 12), ('refugee', 10)],
 [('politics', 7), ('party', 6), ('farron', 5)],
 [('medium', 24), ('candidate', 18), ('macron', 15)]]

The output is a list for each document (this means the output is aligned with the document labels `doc_labels`), with three pairs of `(token, frequency)` each. It's also possible to get this data as datatable via [sorted_terms_data_table()](api.rst#tmtoolkit.bow.bow_stats.sorted_terms_data_table), which gives a better overview and also includes labels for the documents. It accepts the same parameters for sorting and limitting the results:

In [18]:
from tmtoolkit.bow.bow_stats import sorted_terms_data_table

sorted_terms_data_table(dtm, vocab, doc_labels, top_n=3)

,doc,token,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,NewsArticles-119,derbyshire,2
1,NewsArticles-119,bbc,2
2,NewsArticles-119,victoria,2
3,NewsArticles-1206,car,3
4,NewsArticles-1206,collision,3
5,NewsArticles-1206,road,3
6,NewsArticles-2058,merkel,14
7,NewsArticles-2058,eu,12
8,NewsArticles-2058,refugee,10


In [19]:
sorted_terms_data_table(dtm, vocab, doc_labels, lo_thresh=5)

,doc,token,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,NewsArticles-2058,merkel,14
1,NewsArticles-2058,eu,12
2,NewsArticles-2058,refugee,10
3,NewsArticles-2058,germany,8
4,NewsArticles-2058,turkey,7
5,NewsArticles-2058,country,7
6,NewsArticles-2058,europe,6
7,NewsArticles-3016,politics,7
8,NewsArticles-3016,party,6


### Term frequency/inverse document frequency transformation (tf-idf)

tfidf

tf_proportions
idf